In [1]:
# Final script: Beam Search Agglomerative vs Standard Agglomerative Clustering (7 datasets, ARI evaluation)
import numpy as np
import pandas as pd
from sklearn.datasets import (
    make_moons, make_circles, make_blobs, make_classification,
    load_iris, load_wine, load_digits
)
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import adjusted_rand_score
from sklearn.metrics import pairwise_distances
from scipy.cluster.hierarchy import linkage, fcluster
import heapq

# BeamAgglo class definition
class BeamAgglo:
    def __init__(self, clusters, history, cost):
        self.clusters = clusters
        self.history = history
        self.cost = cost

    def __lt__(self, other):
        return self.cost < other.cost

# Beam search agglomerative clustering using average linkage
def beam_search_agglomerative_average(X, y_true, beam_width=3, target_k=2):
    n = X.shape[0]
    dist_matrix = pairwise_distances(X)
    np.fill_diagonal(dist_matrix, np.inf)

    initial = BeamAgglo(clusters=[[i] for i in range(n)], history=[], cost=0)
    beam = [initial]

    while True:
        new_beam = []
        for node in beam:
            c = node.clusters
            merge_attempts = 0
            for i in range(len(c)):
                for j in range(i + 1, len(c)):
                    if merge_attempts >= beam_width * 2:
                        break
                    new_clusters = c[:i] + c[i+1:j] + c[j+1:] + [c[i] + c[j]]
                    # Average linkage cost: mean distance between all pairs across clusters
                    pairwise_dists = dist_matrix[np.ix_(c[i], c[j])]
                    merge_cost = np.mean(pairwise_dists)
                    new_cost = node.cost + merge_cost
                    new_history = node.history + [((c[i], c[j]), merge_cost)]
                    new_beam.append(BeamAgglo(new_clusters, new_history, new_cost))
                    merge_attempts += 1

        beam = heapq.nsmallest(beam_width, new_beam)
        if all(len(b.clusters) <= target_k for b in beam):
            break

    best = min(beam, key=lambda b: b.cost)
    final_labels = np.zeros(n, dtype=int)
    for idx, cluster in enumerate(best.clusters):
        final_labels[cluster] = idx
    ari_beam = adjusted_rand_score(y_true, final_labels)

    return final_labels, ari_beam

# Define synthetic and real datasets
synthetic_datasets = {
    "Moons": make_moons(n_samples=200, noise=0.1, random_state=42),
    "Circles": make_circles(n_samples=200, factor=0.5, noise=0.1, random_state=42),
    "Blobs": make_blobs(n_samples=200, centers=4, cluster_std=0.60, random_state=42),
    "Classification": make_classification(n_samples=200, n_features=10, n_informative=5,
                                          n_redundant=0, n_clusters_per_class=1, n_classes=4, random_state=42)
}

real_datasets = {
    "Iris": load_iris(return_X_y=True),
    "Wine": load_wine(return_X_y=True),
    "Digits (subset)": load_digits(n_class=5, return_X_y=True)  # to limit runtime
}

# Evaluate ARI for all datasets
results = []
all_datasets = {**synthetic_datasets, **real_datasets}

for name, (X, y_true) in all_datasets.items():
    X = StandardScaler().fit_transform(X)
    k = len(np.unique(y_true))

    # Standard Agglomerative (Single Linkage)
    Z = linkage(X, method='single')
    labels_std = fcluster(Z, t=k, criterion='maxclust')
    ari_std = adjusted_rand_score(y_true, labels_std)

    # Beam Search (Average Linkage)
    _, ari_beam_avg = beam_search_agglomerative_average(X, y_true, beam_width=3, target_k=k)

    results.append({
        "Dataset": name,
        "Standard Agglomerative ARI": ari_std,
        "Beam Search (Average Linkage) ARI": ari_beam_avg
    })

# Output final dataframe
df_final = pd.DataFrame(results)
print(df_final)

           Dataset  Standard Agglomerative ARI  \
0            Moons                    0.980000   
1          Circles                    0.021135   
2            Blobs                    1.000000   
3   Classification                    0.000184   
4             Iris                    0.558371   
5             Wine                   -0.006814   
6  Digits (subset)                   -0.000002   

   Beam Search (Average Linkage) ARI  
0                           0.515972  
1                          -0.003392  
2                           0.786530  
3                           0.036884  
4                           0.506688  
5                           0.650379  
6                           0.351480  
